# Neural Network - MNIST

## Өгөгдөл унших

In [7]:
import numpy as np
import mnist
import helper
from matplotlib import pyplot as plt
# % matplotlib inline

training_images, training_labels, test_images, test_labels = mnist.load()


## Neural Network класс

In [8]:
class NN:
    def __init__(self, classes, layer_neuron_counts=None):
        self.layer_neuron_counts = layer_neuron_counts
        self.classes = classes
        self.weights = []
        self.biases = []
        self.neurons = []

        if layer_neuron_counts is not None:
            for i in range(1, len(self.layer_neuron_counts)):
                prev_layer_size = self.layer_neuron_counts[i - 1]
                next_layer_size = self.layer_neuron_counts[i]
                self.weights.append(np.random.randn(prev_layer_size, next_layer_size))
                self.biases.append(np.zeros(next_layer_size))

    def train(self, training_datas, training_labels, epoch, batch_count):
        for e in range(1, epoch + 1):
            print('Epoch: ' + str(e))
            batch_size = round(len(training_datas) / batch_count)
            for b in range(0, batch_count):
                helper.print_progress(b + 1, batch_count)
                minibatch_data = training_datas[b * batch_size:(b + 1) * batch_size]
                minibatch_label = training_labels[b * batch_size:(b + 1) * batch_size]
                self.forward(minibatch_data, minibatch_label)
            print('')

    def predict(self, x):
        self.neurons = [x]
        for i, (weights, biases) in enumerate(zip(self.weights, self.biases)):
            if i == len(self.weights) - 1:
                self.neurons.append(self.softmax(self.calculate_layer(self.neurons[-1], weights, biases)))
            else:
                self.neurons.append(self.relu(self.calculate_layer(self.neurons[-1], weights, biases)))
        return self.neurons[-1]

    def calculate_layer(self, x, weight, bias):
        return x @ weight + bias

    def relu(self, h):
        h[h <= 0] = 0
        return h

    def softmax(self, y):
        e = [np.exp(i) for i in y]
        sum_of_e = sum(e)
        return [i / sum_of_e for i in e]

    def forward(self, train_datas, train_labels):
        for i, data in enumerate(train_datas):
            x = self.predict(data)
            labels = helper.ohe(self.classes, training_labels[i])
            nll = -sum(self.loss(x, labels))
        return None

    def loss(self, predicted_labels, labels):
        return np.sum(labels * np.log(predicted_labels))


In [5]:
classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
nn = NN(classes, [len(training_images[0]), 16, len(classes)])
nn.train(training_images, training_labels, 1, 10000)


Epoch: 1


-69.07715095906558
-16.94360046583019
-23.683871817714124
-22.253662490257522
-53.032452837198655
-109.25708179925428


-76.77065414639885
-72.58279102118921
-39.112370465171445
-31.668374560318746
-63.03245114722521
-96.58253988145438


-96.26802214740802
-45.2875269118341
-17.25128389532119
-44.2393968494928
-59.88412175720716
-108.52840054812492


-15.20195768201331
-58.084560790305275
-0.6349479445102123
-24.312696160784927
-55.74232079422087
-89.05816272679465


...

-106.36423406132212


-62.57864725698795
-35.236912629867625
-53.43923344072708
-29.65042870719995
-69.27345942789837
-90.53951020038909


-93.87556973356864
-67.33328895348508
-61.97364064445322


-55.523210101056975
-28.50168187342341
-63.726768616430746


KeyboardInterrupt: 